In [1]:
import numpy as np
import tensorflow as tf
import input_data

In [2]:
mnist = input_data.read_data_sets('MNIST_data' , one_hot = True)

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


In [3]:
lr = 0.001
training_iters = 1000
batch_size = 128#mini batch size 128张图片
display_step = 10

n_inputs = 28 #28列
n_steps = 28 #时间步
n_hidden_units = 128
n_classes = 10

In [4]:
x = tf.placeholder(tf.float32 , [None , n_steps , n_inputs])
y = tf.placeholder(tf.float32 , [None , n_classes])

In [5]:
weights = {'in':tf.Variable(tf.random_normal([n_inputs , n_hidden_units])),
          'out':tf.Variable(tf.random_normal([n_hidden_units , n_classes]))}

biases = {'in':tf.Variable(tf.constant(0.1 , shape = [n_hidden_units ,])),
         'out':tf.Variable(tf.constant(0.1 , shape = [n_classes]))}

In [6]:
def RNN(X , weights , biases):
    X = tf.reshape(X , [-1 , n_inputs])
    X_in = tf.matmul(X , weights['in']) + biases['in']
    X_in = tf.reshape(X_in , [-1 , n_steps , n_hidden_units])
    
    lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(n_hidden_units , forget_bias=1.0 , state_is_tuple=True)
    #lstm_cell = tf.contrib.rnn.BasicLSTMCell(num_units=n_hidden_units)
    _init_state = lstm_cell.zero_state(batch_size , dtype = tf.float32)
    
    #1 lstm cell #input #initial state #这个是时间步 是以哪个为准
    #2 states 为 tuple 在LSTM中 tuple是cstate mstate(output) 在RNN中 只是mstate
    #3 time_major 表示时间步是不是以X_in的第0个维度（主要维度） 还是第1个维度（次要维度s）
    #4 每一步的结果 都存在outputs中  所以outputs是tensor类型
    #5 states是一个tuple 保存最后一个时间步的信息
    #6 states[1] 即outputs的最后一个(先要将tensor转换为list)
        #states[1]=outputs[-1]
    #7 outputs即每一个时间步的输出（mstate）
    
    outputs , states = tf.nn.dynamic_rnn(lstm_cell , X_in , initial_state=_init_state , time_major=False)
    #outputs , states = tf.nn.dynamic_rnn(lstm_cell , X_in , initial_state=None , dtype=tf.float32 , time_major=False)
    
    results = tf.matmul(states[1] , weights['out']) + biases['out']
    
    return results

In [ ]:
pred = RNN(x , weights , biases)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y , logits=pred))
train_step = tf.train.AdamOptimizer(lr).minimize(cost)

correct_pred = tf.equal(tf.argmax(pred , 1) , tf.argmax(y , 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred , tf.float32))

init = tf.global_variables_initializer()


with tf.Session() as sess:
    sess.run(init)
    step = 0
    
    while step*batch_size < training_iters:
        batch_xs , batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape([batch_size , n_steps , n_inputs])
        sess.run([train_step] , feed_dict={x:batch_xs , y:batch_ys})
        
        if step %20 == 0:
            print(sess.run(accuracy , feed_dict={x:batch_xs , y:batch_ys}))
            

0.21875
0.1875
0.296875
0.335938
0.4375
0.507813
0.539063
0.554688
0.617188
0.648438
0.6875
0.703125
0.648438
0.640625
0.523438
0.601563
0.632813
0.617188
0.617188
0.617188
0.601563
0.609375
0.664063
0.75
0.78125
0.6875
0.679688
0.726563
0.703125
0.703125
0.671875
0.796875
0.757813
0.796875
0.648438
0.695313
0.804688
0.773438
0.765625
0.765625
0.6875
0.625
0.773438
0.78125
0.773438
0.6875
0.75
0.75
0.734375
0.75
0.765625
0.796875
0.757813
0.8125
0.78125
0.875
0.789063
0.773438
0.765625
0.6875
0.78125
0.796875
0.726563
0.695313
0.867188
0.8125
0.835938
0.835938
0.78125
0.796875
0.734375
0.757813
0.757813
0.765625
0.757813
0.664063
0.710938
0.851563
0.8125
0.859375
0.851563
0.8125
0.820313
0.804688
0.765625
0.796875
0.8125
0.859375
0.890625
0.90625
0.804688
0.796875
0.78125
0.84375
0.859375
0.820313
0.835938
0.84375
0.8125
0.789063
0.773438
0.867188
0.914063
0.875
0.765625
0.828125
0.867188
0.84375
0.890625
0.875
0.84375
0.90625
0.851563
0.875
0.929688
0.90625
0.875
0.859375
0.835938
0.8

0.96875
0.960938
0.953125
0.953125
0.945313
0.976563
0.976563
0.984375
0.992188
0.984375
0.96875
0.984375
1.0
0.992188
0.992188
0.976563
0.960938
0.976563
0.992188
0.992188
0.984375
1.0
0.976563
0.984375
0.976563
0.992188
0.976563
1.0
0.976563
0.984375
0.96875
0.96875
0.992188
0.96875
0.960938
0.976563
0.984375
0.945313
0.945313
0.945313
0.96875
0.976563
0.96875
0.96875
0.960938
0.984375
0.976563
0.976563
0.960938
0.96875
0.960938
0.976563
0.96875
0.984375
0.945313
0.976563
0.992188
0.953125
0.945313
0.96875
0.976563
0.984375
0.976563
0.96875
0.976563
0.976563
0.945313
0.9375
0.96875
0.984375
0.953125
1.0
0.992188
0.984375
0.96875
0.96875
0.984375
0.984375
0.96875
0.976563
0.984375
0.96875
0.96875
0.960938
0.960938
0.992188
0.960938
0.96875
0.976563
0.992188
0.976563
1.0
0.960938
0.976563
0.96875
0.96875
1.0
0.976563
0.984375
0.976563
0.992188
0.976563
1.0
0.992188
0.96875
0.976563
0.984375
0.976563
0.976563
0.976563
0.984375
0.984375
0.984375
0.960938
0.976563
0.984375
0.953125
0.9765

1.0
0.976563
0.976563
0.984375
0.992188
0.984375
0.984375
0.984375
0.984375
0.992188
0.976563
0.992188
0.992188
0.984375
0.96875
1.0
0.992188
1.0
0.984375
0.984375
0.992188
0.96875
0.984375
0.992188
0.984375
0.992188
1.0
0.992188
0.976563
0.984375
0.992188
0.984375
0.992188
0.96875
0.992188
0.992188
0.992188
0.992188
0.992188
0.976563
1.0
1.0
1.0
0.984375
0.976563
0.960938
0.976563
0.992188
0.960938
0.976563
1.0
0.992188
0.984375
0.976563
0.976563
0.992188
0.984375
1.0
1.0
0.992188
1.0
0.992188
0.96875
0.992188
0.992188
0.976563
0.992188
0.992188
0.992188
1.0
0.984375
0.984375
0.992188
1.0
1.0
0.984375
0.992188
0.96875
0.992188
0.992188
1.0
0.976563
0.953125
0.984375
0.976563
0.992188
0.992188
1.0
0.992188
0.992188
0.992188
1.0
0.984375
0.992188
0.992188
0.984375
0.984375
0.992188
0.992188
0.992188
0.984375
0.992188
0.992188
1.0
0.984375
0.976563
0.992188
0.953125
0.992188
0.984375
1.0
0.984375
1.0
1.0
0.984375
0.984375
0.976563
0.992188
0.976563
1.0
0.992188
0.960938
0.976563
0.984375

0.992188
1.0
0.992188
0.992188
1.0
0.992188
0.992188
0.992188
1.0
0.992188
0.984375
0.992188
1.0
1.0
0.992188
0.992188
0.992188
1.0
0.992188
0.992188
0.984375
0.984375
1.0
1.0
0.984375
1.0
0.992188
1.0
1.0
0.992188
1.0
0.992188
0.984375
1.0
0.992188
0.984375
0.976563
1.0
0.976563
1.0
0.992188
0.992188
1.0
1.0
0.992188
0.984375
1.0
0.984375
1.0
0.984375
1.0
1.0
1.0
0.984375
1.0
0.992188
1.0
0.992188
1.0
0.992188
0.984375
0.992188
0.992188
0.976563
0.984375
0.984375
1.0
0.992188
1.0
1.0
0.992188
0.992188
0.992188
1.0
1.0
1.0
1.0
0.992188
1.0
1.0
1.0
0.992188
0.992188
0.992188
0.992188
1.0
0.984375
1.0
1.0
0.992188
0.992188
0.992188
0.992188
0.976563
0.992188
1.0
1.0
1.0
1.0
0.992188
1.0
0.992188
1.0
1.0
1.0
0.992188
1.0
0.992188
1.0
0.992188
0.992188
0.992188
0.992188
1.0
0.992188
0.992188
1.0
1.0
1.0
0.992188
0.992188
0.992188
0.984375
1.0
0.992188
1.0
0.992188
1.0
1.0
1.0
1.0
1.0
1.0
0.992188
0.984375
1.0
0.992188
0.992188
1.0
0.992188
0.992188
0.992188
1.0
1.0
0.992188
0.992188
0.9843